In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd '/content/drive/My Drive/Proyecto 2 SENSORES/Ficheros de datos/'

/content/drive/My Drive/Proyecto 2 SENSORES/Ficheros de datos


In [0]:
!pip install pyproj

In [0]:
#SE IMPORTAN LAS LIBRERÍAS NECESARIAS PARA EL FICHERO
from pyproj import Proj, transform
from pyproj import Transformer
import datetime as dt
from datetime import datetime
import numpy as np
import json,csv
import os
import pandas as pd 
import matplotlib.pyplot as plt
import math

In [0]:
#Mediante la función "timeFormated" se pasa el formato de la variable "Timestamp" a formato de hora normal.
def timeFormated(date):
  dt_string = datetime.strptime(str(date),"%Y-%m-%dT%H:%M:%S+01:00") 
  return dt_string.strftime("%Y-%m-%d %H:%M:%S")

In [0]:
#Mediante la función "coordsColumnsTransformation" se pasa el formato de las coordenadas que aparece inicialmente a coordenadas normales
#de latitud y longitud
def coordsColumnsTransformation(data):
  transformer = Transformer.from_crs('epsg:25830','epsg:4326',always_xy=True)
  puntos = list(zip(data.ycoord, data.xcoord))
  coords = np.array(list(transformer.itransform(puntos)))
  data = data.rename(columns=({'ycoord':'latitud','xcoord':'longitud'}))
  data['longitud'] = coords[:,0]
  data['latitud'] = coords[:,1]
  return data

#coordsColumnsTransformation(data)

In [0]:
#Mediante la función "distance2upv" se obtiene el sensor más cercano a la UPV

def distance2upv(data):
  puntos = list(zip(data.latitud, data.longitud))
  #coords = np.array(list(transformer.itransform(puntos)))
  dist = list(map(distanceOriginal,puntos))
  data['distance2upv'] = dist
  return data



In [0]:
# Calcula la distancia de un par de coordenadas a el sensor nuestro de la UPV
def distanceOriginal(tupla):
  lat,lon = tupla
  upvLat,upvLon = (39.482921, -0.348383)
  dist = math.sqrt( (upvLat - lat)**2 + (upvLon - lon)**2 )
  return dist

In [0]:
#Mediante la función "espirasToCsv" se pasan los JSON relativos a las espiras a formato CSV para posteriores análisis
def espirasToCsv(dir = './espirasCesar/'):
  ### Lee los archivos json de la carpeta espiras situada en Drive y los mete todos
  ### en un archivo .csv
  espiras_totales = []
  fileNames = os.listdir(dir)
  #Para cada archivo json lo abrimos, lo procesamos y lo añadimos a un csv
  for fileJson in fileNames:
    with open (dir+fileJson,mode='r',encoding='UTF-8') as f:
      data = json.load(f)
      espiras = data['resources']
    ## Eliminamos el campo 'uri' que no nos interesa
    for espira in espiras:          #Espiras es una lista de diccionarios
      del espira['uri']
    espiras_totales.extend(espiras) # Agregamos a las espiras totales el contenido de espiras 
    #print(len(espiras_totales))                                    # solo los diccionarios

  csvAllDicts = 'espiras.csv'
  keys = espiras_totales[0].keys()

  with open (csvAllDicts,mode='w',encoding='UTF-8') as f:
      dict_writer = csv.DictWriter(f, keys)
      dict_writer.writeheader()
      dict_writer.writerows(espiras_totales)
  data = pd.read_csv("espiras.csv")
  index_names = data[data['sensor']== int('3022')].index    #Seleccionamos las filas que tienen el sensor 3022 que está mal
  data.drop(index_names, inplace=True)                      #Eliminamos las filas que tienen el sensor malo
  index_names = data[data['sensor']== int('3023')].index    #Seleccionamos las filas que tienen el sensor 3023 que está mal
  data.drop(index_names, inplace=True)
  data['modified'] = data['modified'].map(timeFormated)     #Cambiamos la fecha de datos a la fecha formateada
  data = coordsColumnsTransformation(data)                  #Transformamos las coordenadas a formato latitud, longitud
  data = distance2upv(data)
  data.to_csv(csvAllDicts, sep=',', encoding='utf-8',index=False)

espirasToCsv()

In [0]:
espirasToCsv(dir = "./espirasPrueba/")
data = pd.read_csv('espiras.csv')
data.nsmallest(5, ['distance2upv'])

,modified,intensidad,angulo,latitud,longitud,sensor,distance2upv
60,2020-03-04 13:11:01,300,251,39.478330,-0.345866,2041,0.005236
78,2020-03-04 13:11:01,90,254,39.477467,-0.346250,2043,0.005856
71,2020-03-04 13:11:01,330,75,39.477107,-0.346630,2042,0.006073
52,2020-03-04 13:11:01,360,342,39.474269,-0.348575,2032,0.008654
54,2020-03-04 13:11:01,90,160,39.473443,-0.347090,2033,0.009566


In [0]:
"""
Graficamos los sensores más cercanos, y niguno cae cerca de la UPV.
"""

datosFirebase.csv   espiras.csv		      firebaseAnomalos.json
datosFirebase.json  ficherosEspirasFerri.zip  intensidadEspiras
espiras		    firebaseAnomalos.csv      tramos


In [0]:
#Una vez graficada el sensor más cercano podemos ver que está muy lejos de
#nuestro sensor.
"""Esta función guarda las coordenadas del tramo indicado en un csv
  que posteriormente es importado a Google Maps y es mostrado ahí
"""
def coordSensor():
  row =data.loc[data['sensor'] ==sensor]
  coordenadas = row['coordinates'].tolist()

  coordenadas =  ast.literal_eval(coordenadas[0])
  filepath = './coordenadasTramos/'
  with open (filepath +sensor+'.csv',mode='w',encoding='UTF-8') as file:
        file_writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for i in coordenadas:
            file_writer.writerow(i)

coordSensor()